# Fetching Cropped GOES16 Training Samples

Now we have all of the pieces to use the IBTrACS data to crop a GOES16 image around the tropical cyclone. To overcome the issue of nighttime images, we'll use band 7 of the ABI. This band is the shortwave IR band, with central wavelength at 3.9 microns.

Downloading all of these GOES images will take some time, but I think it would be best to have an equal proportion of positive and negative samples in the training set. Fortunately, tropical cyclones aren't outrageously large, so we can use different parts of the same GOES image for positive and negative samples. This does have the problem though of statistically favoring images from the typical hurricane season, and this may need to be addressed later.

## Import Modules

We start by importing any of the needed modules. Let's also avoid plotting each image.

In [1]:
# Import modules
import pandas as pd
import datetime
import numpy as np
from pyproj import Proj
import matplotlib
import matplotlib.pyplot as plt
import time
import warnings
import os

# Import functions I've written
os.chdir("..")
import goes
import ibtracs

# Don't display plots or warnings in the notebook
matplotlib.use('Agg')
warnings.filterwarnings('ignore')

## Load Valid IBTrACS Data

We've already gone through and removed any IBTrACS data that wasn't valid for our training purposes. Let's also filter out the storm's *nature*. The nature of the storm has a few options: 
* TS - storm is tropical 
* SS - Subtropical 
* ET - Extratropical 
* DS - Disturbance 
* MX - Mix of conflicting reports 
* NR - Not Reported 
* MM - Missing

For now, let's just focus on the tropical storms. Lastly, create a column that we'll set to `True` as we save off the images. Also, let's only look at the year 2017 for now.

In [2]:
df = ibtracs.read_data('./data/ibtracs_GOES16.csv',subsetSeason=True,yearStart=2017,yearEnd=2017)
df = df[df['NATURE']=='TS'].reset_index()

I don't want to run this notebook for an entire year's worth of data, as that will probably take a while. Let's subset the data even further.

In [3]:
df = df[(df['ISO_TIME'] > datetime.datetime(2017,8,25)) & (df['ISO_TIME'] < datetime.datetime(2017,8,26))].copy()
df.head()

,index,SID,SEASON,NUMBER,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,TRACK_TYPE,DIST2LAND,LANDFALL,IFLAG,STORM_SPEED,STORM_DIR
1605,2197,2017228N14314,2017,61,HARVEY,2017-08-25 03:00:00,TS,25.2924,-94.7578,,,main,243,204,P_____________,9,313
1606,2198,2017228N14314,2017,61,HARVEY,2017-08-25 06:00:00,TS,25.6000,-95.1000,90,966,main,204,170,O_____________,9,315
1607,2199,2017228N14314,2017,61,HARVEY,2017-08-25 09:00:00,TS,25.9350,-95.4651,,,main,160,133,P_____________,9,318
1608,2200,2017228N14314,2017,61,HARVEY,2017-08-25 12:00:00,TS,26.3000,-95.8000,95,949,main,133,123,O_____________,9,325
1609,2201,2017228N14314,2017,61,HARVEY,2017-08-25 15:00:00,TS,26.6999,-96.0652,,,main,126,108,P_____________,9,331


## Define Custom Functions

These are just quick functions to save the image centered about the specified location.

In [4]:
def crop_image(ds,xInd,yInd,buffer,myDPI,figPath):
    
    '''From a GOES image, crop an image centered around [yInd,xInd] of size 2*buffer by 2*buffer'''
    
    f = plt.figure(frameon=False)
    f.set_size_inches(buffer*2/myDPI,buffer*2/myDPI)
    plt.imshow(ds.Rad[yInd-buffer:yInd+buffer,xInd-buffer:xInd+buffer],cmap='gray_r');
    plt.axis('off');
    plt.savefig(figPath, facecolor='w', dpi=220, edgecolor='w', bbox_inches='tight', pad_inches=0);
    f.clear();
    plt.close(f);
    del(f)

## Crop Training Examples

Now let's loop through the dataframe observations and save off cropped examples as positive or negative training samples. First, we get all of the unique times in the data frame, and for each unique time, we find all of the IBTrACS observations that match that time. Then, we grab the image associated with that time and crop out all of the IBTrACS observations by some set buffer size about the central IBTrACS location. We then identify the pixels we saved off so that we can keep track of what pixels are free to take negative samples from in the future. Once we've done all of the positive training samples from the IBTrACS unique time for an image, we then randomly select a pixel location, test if that proposed cropped image overlaps with previously taken images, and then save the image if it doesn't overlap with other samples.

In [9]:
# Write out a log file
logFile = open("training.log",'w')

# Get the starting time
tic = time.perf_counter()
count = 0

# Set the buffer (in pixels) size
myDPI = 166
buffer = int(myDPI*1.5)

# Loop through all of the dates in the IBTrACS dataframe
for date in df['ISO_TIME'].unique():
    
    # Update log file
    logFile.write("Date: "+str(date)+"\n")

    # Convert the numpy.datetime64 object into a datetime object
    dt = datetime.datetime.fromisoformat(str(date)[:-3])

    # Update log file
    logFile.write("\tDownloading imagery\n")
        
    try:

        # Get the image for the specified date
        ds = goes.download_data(date=dt,credPath="secrets.csv",bucketName="noaa-goes16",product='ABI-L1b-RadF',band=13)

        # Update log file
        logFile.write("\tProjecting data\n")

        # Get dataset projection data
        satHeight = ds.goes_imager_projection.perspective_point_height
        satLon = ds.goes_imager_projection.longitude_of_projection_origin
        satSweep = ds.goes_imager_projection.sweep_angle_axis
        majorMinorAxes = (ds.goes_imager_projection.semi_major_axis,ds.goes_imager_projection.semi_minor_axis)

        # The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
        x = ds.variables['x'][:] * satHeight
        y = ds.variables['y'][:] * satHeight

        # Create an array of ones for the negative training samples
        nx, ny = ds.Rad.shape
        free = np.ones((ny,nx))

        # Create a pyproj geostationary map object
        p = Proj(proj='geos', h=satHeight, lon_0=satLon, sweep=satSweep)

        # Get the subset of the dataframe that matches this date
        dfSubset = df[df['ISO_TIME'] == date]
        indices = dfSubset.index.values

        # Update log file
        logFile.write("\tSaving cropped images\n")

        # Loop through all of the subset dataframe rows:
        for i in indices:

            # Get the current row
            row = dfSubset[dfSubset.index==i]

            # Get the latitudes/longitudes of the track data
            trackLat = float(row['LAT'])
            trackLon = float(row['LON'])

            # Convert lon/lat to x/y
            trackX,trackY = p(trackLon,trackLat)

            # Get the closest point to the IBTrACS data
            xInd = np.nanargmin(abs(x-trackX))
            yInd = np.nanargmin(abs(y-trackY))

            # Set the figure file name
            figPart = dt.strftime("%Y%m%d_%HZ")+f'_{xInd:05.0f}_{yInd:05.0f}_{buffer}'
            figPath = f'./training-data/images/positive/{figPart}_cropped.png'

            # Save the cropped image
            crop_image(ds,xInd,yInd,buffer,myDPI,figPath)

            # Mark these pixels as containing a positive sample
            free[yInd-buffer:yInd+buffer,xInd-buffer:xInd+buffer] = 0

        # Now go find some random negative samples that don't overlap with locations where free == 0
        negativeCount = 0
        maxAttemptCount = 5*len(indices)
        attemptCount = 0
        while negativeCount < len(indices):

            # Get a random location on the image
            xInd = np.random.randint(buffer,nx-buffer,size=1)[0]
            yInd = np.random.randint(buffer,ny-buffer,size=1)[0]

            # Check that it doesn't overlap with previous images
            if (free[yInd-buffer:yInd+buffer,xInd-buffer:xInd+buffer] == 1).all():

                # Set the figure file name
                figPart = dt.strftime("%Y%m%d_%HZ")+f'_{xInd:05.0f}_{yInd:05.0f}_{buffer}'
                figPath = f'./training-data/images/negative/{figPart}_cropped.png'

                # Save the cropped image
                crop_image(ds,xInd,yInd,buffer,myDPI,figPath)

                # Also mark these pixels, since we have a sample already taken from them
                free[yInd-buffer:yInd+buffer,xInd-buffer:xInd+buffer] = 0

                # Advance the count of negative training sample images obtained
                negativeCount = negativeCount + 1

            # Advance the count of attempts and break if it has been going for too long
            attemptCount = attemptCount + 1
            if attemptCount > maxAttemptCount:
                break

        # Delete the dataset to save memory
        del(ds)

    except:
        logFile.write("\tError saving image\n")
        
# Get the ending time
toc = time.perf_counter()

# Close the logfile and write out the total time
logFile.write("\n\n Total time taken: "+str(toc-tic))
logFile.close()